The recommended starting point for all manipulation of Neuropixels data stored in NWB files is pynapple.

https://pynapple.org/index.html#pynapple-python-neural-analysis-package

Before starting this tutorial, you should read all the documentation there.

You can use the python interpreter /scratch/qgreicius/conda_envs/nwb/bin/python for this demo.

In [1]:
import pynapple as nap
from pathlib import Path

In [2]:

nwb_path = Path("/data_store2/neuropixels/nwb/NP128_B1.nwb")
data = nap.load_file(nwb_path)


In [3]:
print(data)
spike_times = data["NP128_B1_g0_imec0_KS4"]

NP128_B1
┍━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━┑
│ Keys                        │ Type        │
┝━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━┥
│ TaskTimes                   │ IntervalSet │
│ TaskStims                   │ IntervalSet │
│ StimSpeechWords             │ IntervalSet │
│ StimSpeechTrials            │ IntervalSet │
│ StimSpeechPhones            │ IntervalSet │
│ ProdSpeechWords             │ IntervalSet │
│ ProdSpeechTrials            │ IntervalSet │
│ ProdSpeechPhones            │ IntervalSet │
│ KilosortSortTimes           │ IntervalSet │
│ pitch                       │ TsdFrame    │
│ intensity                   │ TsdFrame    │
│ artics_new                  │ TsdFrame    │
│ NP128_B1_g0_imec1_KS4_Th=12 │ TsGroup     │
│ NP128_B1_g0_imec1_KS4       │ TsGroup     │
│ NP128_B1_g0_imec0_KS4_Th=12 │ TsGroup     │
│ NP128_B1_g0_imec0_KS4       │ TsGroup     │
│ TimeSeriesNIDQ              │ TsdFrame    │
│ ElectricalSeriesLFImec1     │ TsdFrame    │
│ ElectricalSeriesLFImec0

/userdata/ekato/miniforge3/envs/se2nwb/lib/python3.10/site-packages/pynapple/core/base_class.py:60: UserWarning: Some epochs have no duration
  self.time_support = IntervalSet(start=self.index[0], end=self.index[-1])
/userdata/ekato/miniforge3/envs/se2nwb/lib/python3.10/site-packages/pynapple/core/base_class.py:62: RuntimeWarning: divide by zero encountered in scalar divide
  self.rate = self.index.shape[0] / np.sum(


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# print(spike_times)
KSLabels = spike_times.metadata["KSLabel_repeat"]
firingRates = spike_times.metadata["rate"]
contamPct = spike_times.metadata["ContamPct"]

mask1 = firingRates>0.5
mask2 = contamPct<10
mask3 = KSLabels=="good"
combined_mask = mask1 & mask2 & mask3
indices = firingRates[combined_mask].index

# now apply ISI violation criteria to narrow down good neurons
violationThreshold = 3/1000 #for 3 ms isi refractory period, lit standard
violationPct = np.zeros(len(indices))
ct = 0;
for i in indices:
    unit = spike_times[i]
    unit = unit.as_series().index
    isi = unit.diff()[1:len(unit)]
    violations = np.where(isi<violationThreshold)
    violations = np.array(violations)
    violationPct[ct] = violations.size/len(isi)
    ct += 1
# plt.hist(violationPct, bins = 20)
mask = violationPct<5/100 # throwing out neurons with > 3% isi violations
indicesFinal = indices[mask]
print(len(indicesFinal))


spike_times_good = spike_times[indicesFinal]
counts = spike_times_good.count(bin_size = 1*60*10)
print(counts)

start_times = [unit.as_series().index.values[0] for unit in spike_times_good.values()]
print(f"Start times for {len(start_times)} units:")
for i, start_time in enumerate(start_times):
    print(f"  Unit {i}: {start_time:.3f}s")

# for the full run need to confirm the start times are ok across units and write
# the whole thing as a .py script

20
Time (s)        26    30    45    49    63    ...
--------------  ----  ----  ----  ----  ----  -----
1096.648530984  1908  864   2763  1045  2142  ...
1696.648530984  433   257   13    17    3     ...
2296.648530984  6     63    3     0     0     ...
Metadata
unit_name       26    30    45    49    63    ...
KSLabel         good  good  good  good  good  ...
...             ...   ...   ...   ...   ...   ...
dtype: int64, shape: (3, 20)
Start times for 20 units:
  Unit 0: 797.065s
  Unit 1: 822.763s
  Unit 2: 799.697s
  Unit 3: 796.904s
  Unit 4: 796.826s
  Unit 5: 796.657s
  Unit 6: 798.753s
  Unit 7: 796.680s
  Unit 8: 798.068s
  Unit 9: 796.822s
  Unit 10: 796.682s
  Unit 11: 796.689s
  Unit 12: 797.244s
  Unit 13: 796.649s
  Unit 14: 796.668s
  Unit 15: 801.441s
  Unit 16: 797.001s
  Unit 17: 797.305s
  Unit 18: 796.676s
  Unit 19: 797.166s


In [5]:
# Import the exact Harris et al. 2003 GLM implementation
from harris_2003_glm_timescale import (
    run_harris_glm_timescale_analysis, 
    plot_harris_glm_results, 
    find_optimal_timescale_parameters
)

# Run the analysis with the exact Harris et al. 2003 method
results = run_harris_glm_timescale_analysis(
    spike_times,
    indicesFinal, 
    fast_mode=False,
    cv_enabled=False,
    )

# Plot 2D results showing σ vs dt parameter space
plot_harris_glm_results(results)

# Find optimal timescale parameters
find_optimal_timescale_parameters(results)

🐌 FULL MODE: Using comprehensive parameter space
🚀 Starting Harris et al. 2003 GLM-based optimal timescale analysis...
📊 Testing Gaussian widths (σ): [10, 25, 50, 100, 200, 500] ms
⏱️  Testing time bin sizes (dt): [5] ms
🧠 Number of good neurons: 20
🔢 Total parameter combinations: 6

📖 This implements the exact method from Harris et al. 2003:
   - Gaussian smearing of peer cell spike trains
   - Piecewise link function g(η) = {exp(η) if η<0, η+1 if η≥0}
   - Penalized log-likelihood optimization

⚡ OPTIMIZATIONS:
   - Vectorized Gaussian smearing
   - Reduced optimization iterations
   - Time point subsampling if needed
   - Cross-validation disabled (faster execution)

Starting analysis of 6 parameter combinations...

[1/6] (16.7%) Analyzing Harris GLM: σ=10.0ms, dt=5.0ms
  Elapsed: 0.0s, ETA: 0.0s
    Analyzing 20 neurons...
      🔍 DEBUG: Starting prediction for neuron 0
      🔍 DEBUG: Target neuron has 2348 spikes
      🔍 DEBUG: Peer neuron 30 has 1192 spikes
      🔍 DEBUG: Peer ne

Traceback (most recent call last):
  File "/userdata/gumbach/git_repos/tumor_np/harris_2003_glm_timescale.py", line 404, in predict_single_neuron_harris_glm
    r2 = r2_score(target_spikes, predicted_intensity)
  File "/userdata/ekato/miniforge3/envs/se2nwb/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 218, in wrapper
    return func(*args, **kwargs)
  File "/userdata/ekato/miniforge3/envs/se2nwb/lib/python3.10/site-packages/sklearn/metrics/_regression.py", line 1276, in r2_score
    _check_reg_targets_with_floating_dtype(
  File "/userdata/ekato/miniforge3/envs/se2nwb/lib/python3.10/site-packages/sklearn/metrics/_regression.py", line 209, in _check_reg_targets_with_floating_dtype
    y_type, y_true, y_pred, sample_weight, multioutput = _check_reg_targets(
  File "/userdata/ekato/miniforge3/envs/se2nwb/lib/python3.10/site-packages/sklearn/metrics/_regression.py", line 116, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
  Fi

      ⚠️  WARNING: Peer neuron 5 has no spikes in time window
      ⚠️  WARNING: Peer neuron 7 has no spikes in time window
      ❌ ERROR: GLM failed with exception: Input contains NaN.
      🔍 DEBUG: Full traceback:
      ❌ Neuron 1: Failed prediction
      🔍 DEBUG: Starting prediction for neuron 2
      🔍 DEBUG: Target neuron has 2779 spikes
      🔍 DEBUG: Peer neuron 26 has 2348 spikes
      🔍 DEBUG: Peer neuron 30 has 1192 spikes
      🔍 DEBUG: Peer neuron 49 has 1062 spikes
      🔍 DEBUG: Peer neuron 63 has 2145 spikes
      🔍 DEBUG: Peer neuron 72 has 20139 spikes
      🔍 DEBUG: Peer neuron 96 has 1187 spikes
      🔍 DEBUG: Peer neuron 107 has 4140 spikes
      🔍 DEBUG: Peer neuron 108 has 1064 spikes
      🔍 DEBUG: Peer neuron 109 has 2025 spikes
      🔍 DEBUG: Peer neuron 113 has 28514 spikes
      🔍 DEBUG: Peer neuron 117 has 2482 spikes
      🔍 DEBUG: Peer neuron 131 has 1619 spikes
      🔍 DEBUG: Peer neuron 138 has 1059 spikes
      🔍 DEBUG: Peer neuron 144 has 17848 spikes

Traceback (most recent call last):
  File "/userdata/gumbach/git_repos/tumor_np/harris_2003_glm_timescale.py", line 404, in predict_single_neuron_harris_glm
    r2 = r2_score(target_spikes, predicted_intensity)
  File "/userdata/ekato/miniforge3/envs/se2nwb/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 218, in wrapper
    return func(*args, **kwargs)
  File "/userdata/ekato/miniforge3/envs/se2nwb/lib/python3.10/site-packages/sklearn/metrics/_regression.py", line 1276, in r2_score
    _check_reg_targets_with_floating_dtype(
  File "/userdata/ekato/miniforge3/envs/se2nwb/lib/python3.10/site-packages/sklearn/metrics/_regression.py", line 209, in _check_reg_targets_with_floating_dtype
    y_type, y_true, y_pred, sample_weight, multioutput = _check_reg_targets(
  File "/userdata/ekato/miniforge3/envs/se2nwb/lib/python3.10/site-packages/sklearn/metrics/_regression.py", line 116, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
  Fi

      ⚠️  WARNING: Peer neuron 3 has no spikes in time window
      ⚠️  WARNING: Peer neuron 4 has no spikes in time window
      ⚠️  WARNING: Peer neuron 5 has no spikes in time window
      ⚠️  WARNING: Peer neuron 7 has no spikes in time window
      ❌ ERROR: GLM failed with exception: Input contains NaN.
      🔍 DEBUG: Full traceback:
      ❌ Neuron 5: Failed prediction
      🔍 DEBUG: Starting prediction for neuron 6
      🔍 DEBUG: Target neuron has 1187 spikes
      🔍 DEBUG: Peer neuron 26 has 2348 spikes
      🔍 DEBUG: Peer neuron 30 has 1192 spikes
      🔍 DEBUG: Peer neuron 45 has 2779 spikes
      🔍 DEBUG: Peer neuron 49 has 1062 spikes
      🔍 DEBUG: Peer neuron 63 has 2145 spikes
      🔍 DEBUG: Peer neuron 72 has 20139 spikes
      🔍 DEBUG: Peer neuron 107 has 4140 spikes
      🔍 DEBUG: Peer neuron 108 has 1064 spikes
      🔍 DEBUG: Peer neuron 109 has 2025 spikes
      🔍 DEBUG: Peer neuron 113 has 28514 spikes
      🔍 DEBUG: Peer neuron 117 has 2482 spikes
      🔍 DEBUG: Pe

...

Traceback (most recent call last):
  File "/userdata/gumbach/git_repos/tumor_np/harris_2003_glm_timescale.py", line 404, in predict_single_neuron_harris_glm
    r2 = r2_score(target_spikes, predicted_intensity)
  File "/userdata/ekato/miniforge3/envs/se2nwb/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 218, in wrapper
    return func(*args, **kwargs)
  File "/userdata/ekato/miniforge3/envs/se2nwb/lib/python3.10/site-packages/sklearn/metrics/_regression.py", line 1276, in r2_score
    _check_reg_targets_with_floating_dtype(
  File "/userdata/ekato/miniforge3/envs/se2nwb/lib/python3.10/site-packages/sklearn/metrics/_regression.py", line 209, in _check_reg_targets_with_floating_dtype
    y_type, y_true, y_pred, sample_weight, multioutput = _check_reg_targets(
  File "/userdata/ekato/miniforge3/envs/se2nwb/lib/python3.10/site-packages/sklearn/metrics/_regression.py", line 116, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
  Fi

      ⚠️  WARNING: Peer neuron 3 has no spikes in time window
      ⚠️  WARNING: Peer neuron 4 has no spikes in time window
      ⚠️  WARNING: Peer neuron 6 has no spikes in time window
      ⚠️  WARNING: Peer neuron 8 has no spikes in time window
      ❌ ERROR: GLM failed with exception: Input contains NaN.
      🔍 DEBUG: Full traceback:
      ❌ Neuron 10: Failed prediction
      🔍 DEBUG: Starting prediction for neuron 11
      🔍 DEBUG: Target neuron has 2482 spikes
      🔍 DEBUG: Peer neuron 26 has 2348 spikes
      🔍 DEBUG: Peer neuron 30 has 1192 spikes
      🔍 DEBUG: Peer neuron 45 has 2779 spikes
      🔍 DEBUG: Peer neuron 49 has 1062 spikes
      🔍 DEBUG: Peer neuron 63 has 2145 spikes
      🔍 DEBUG: Peer neuron 72 has 20139 spikes
      🔍 DEBUG: Peer neuron 96 has 1187 spikes
      🔍 DEBUG: Peer neuron 107 has 4140 spikes
      🔍 DEBUG: Peer neuron 108 has 1064 spikes
      🔍 DEBUG: Peer neuron 109 has 2025 spikes
      🔍 DEBUG: Peer neuron 113 has 28514 spikes
      🔍 DEBUG: P

Traceback (most recent call last):
  File "/userdata/gumbach/git_repos/tumor_np/harris_2003_glm_timescale.py", line 404, in predict_single_neuron_harris_glm
    r2 = r2_score(target_spikes, predicted_intensity)
  File "/userdata/ekato/miniforge3/envs/se2nwb/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 218, in wrapper
    return func(*args, **kwargs)
  File "/userdata/ekato/miniforge3/envs/se2nwb/lib/python3.10/site-packages/sklearn/metrics/_regression.py", line 1276, in r2_score
    _check_reg_targets_with_floating_dtype(
  File "/userdata/ekato/miniforge3/envs/se2nwb/lib/python3.10/site-packages/sklearn/metrics/_regression.py", line 209, in _check_reg_targets_with_floating_dtype
    y_type, y_true, y_pred, sample_weight, multioutput = _check_reg_targets(
  File "/userdata/ekato/miniforge3/envs/se2nwb/lib/python3.10/site-packages/sklearn/metrics/_regression.py", line 116, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
  Fi

      ⚠️  WARNING: Peer neuron 6 has no spikes in time window
      ⚠️  WARNING: Peer neuron 8 has no spikes in time window


KeyboardInterrupt: 